# 前提条件
pip3 install -U nltk

In [4]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

# Pytorch中的Word2Vec-CBOW和SKIP-GRAM
翻译自https://srijithr.gitlab.io/post/word2vec/ By Srijith Rajamohan博士

## 词嵌入概述
简而言之，单词嵌入是文本的数字表示。它们表示为“ n维”向量，其中维“ n”的数目取决于语料库大小和所需的表达能力。语料库的大小越大，您想要的n就越大。较大的“ n”还允许您捕获嵌入中的更多功能。但是，较大的尺寸涉及更长，更困难的优化过程，因此要使用足够大的“ n”，确定此尺寸通常是特定于问题的。由于研究社区已完成许多工作，因此可以将用于类似问题的维超参数用作基准。

词嵌入在矢量空间中嵌入文本的含义。这意味着含义更近的词（即同义词）具有几乎相同的向量表示形式。换句话说，含义相似的单词在高维向量空间中的距离较小，而不相关的单词则具有较高的距离。Mikolov提出了一种训练浅层网络以创建矢量表示的方法。这些向量的生成在概念上类似于自动编码器如何起作用以创建输入数据的低维表示。

作为预处理的一部分，这需要生成（上下文，目标）对。给定“上下文”，即句子中一定数量的连续词，预测“目标”，即与句子中与其相关联的词/词组。关联的一个示例是在上下文之后或上下文之前的单词。它也可以是n个上下文词之间的词。生成后，将其传递到神经网络进行训练。在可伸缩的基于词嵌入的NLP算法中，诸如负采样之类的优化有助于显着加快计算速度。

这些都是CBOW和SKIP-GRAM方法的实现。这些没有Mikolov引入的层次化softmax，否定采样或频繁单词的二次采样，因此很容易说明或试验基本概念。语料库的标记化也应该在生成嵌入之前进行，但是为了简单起见，这里也将其省略。单词嵌入还可以用于通过对嵌入进行聚类并选择最接近聚类中心的单词来标识主题。

例
有关这些算法的简要概述，请参阅Mikolov的论文“[单词和短语的分布式表示及其组成](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)”和“[向量空间中单词表示的有效估计](https://arxiv.org/pdf/1301.3781.pdf)”。

将以下文本视为我们的整个语料库：

从未经历过贫困的人们可能不容易获得贫困的能力。

## 1. CBOW
![连续词袋.png](https://upload.wikimedia.org/wikipedia/commons/a/a2/Cbow.png)

概要：利用文本预测单词

以下是CBOW算法的Pytorch实现。

# 1.1 导入全部的依赖包

In [5]:
# 作者：Srijith Rajamohan基于Robert Guthrie的工作

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import urllib.request
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import word_tokenize
import sklearn
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances

# 1.2 初始化

In [6]:
torch.manual_seed(1)

CONTEXT_SIZE = 3
EMBEDDING_DIM = 10

test_sentence = """Empathy for the poor may not come easily to people who never experienced it. They may blame the victims and insist their predicament can be overcome through determination and hard work.
But they may not realize that extreme poverty can be psychologically and physically incapacitating — a perpetual cycle of bad diets, health care and education exacerbated by the shaming and self-fulfilling prophecies that define it in the public imagination.
Gordon Parks — perhaps more than any artist — saw poverty as “the most savage of all human afflictions” and realized the power of empathy to help us understand it. It was neither an abstract problem nor political symbol, but something he endured growing up destitute in rural Kansas and having spent years documenting poverty throughout the world, including the United States.
That sensitivity informed “Freedom’s Fearful Foe: Poverty,” his celebrated photo essay published in Life magazine in June 1961. He took readers into the lives of a Brazilian boy, Flavio da Silva, and his family, who lived in the ramshackle Catacumba favela in the hills outside Rio de Janeiro. These stark photographs are the subject of a new book, “Gordon Parks: The Flavio Story” (Steidl/The Gordon Parks Foundation), which accompanies a traveling exhibition co-organized by the Ryerson Image Centre in Toronto, where it opens this week, and the J. Paul Getty Museum. Edited with texts by the exhibition’s co-curators, Paul Roth and Amanda Maddox, the book also includes a recent interview with Mr. da Silva and essays by Beatriz Jaguaribe, Maria Alice Rezende de Carvalho and Sérgio Burgi.
""".split()

# 1.3 相关函数

In [7]:
def get_key(word_id):
    for key,val in word_to_ix.items():
        if(val == word_id):
            print(key)

def cluster_embeddings(filename,nclusters):
    X = np.load(filename)
    kmeans = KMeans(n_clusters=nclusters, random_state=0).fit(X)
    center = kmeans.cluster_centers_
    distances = euclidean_distances(X,center)

    for i in np.arange(0,distances.shape[1]):
        word_id = np.argmin(distances[:,i])
        print(word_id)
        get_key(word_id)

def read_data(file_path):
    tokenizer = RegexpTokenizer(r'\w+')
    data = urllib.request.urlopen(file_path)
    data = data.read().decode('utf8')
    tokenized_data = word_tokenize(data)
    stop_words = set(stopwords.words('english'))
    stop_words.update(['.',',',':',';','(',')','#','--','...','"'])
    cleaned_words = [ i for i in tokenized_data if i not in stop_words ]
    return(cleaned_words)

class CBOWModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOWModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))  # -1表示从数据大小推断出该索引的大小
        #print(np.mean(np.mean(self.linear2.weight.data.numpy())))
        out1 = F.relu(self.linear1(embeds)) # 第一层的输出
        out2 = self.linear2(out1)           # 第二层的输出
        #print(embeds)
        log_probs = F.log_softmax(out2, dim=1)
        return log_probs

    def predict(self,input):
        context_idxs = torch.tensor([word_to_ix[w] for w in input], dtype=torch.long)
        res = self.forward(context_idxs)
        res_arg = torch.argmax(res)
        res_val, res_ind = res.sort(descending=True)
        res_val = res_val[0][:3]
        res_ind = res_ind[0][:3]
        #print(res_val)
        #print(res_ind)
        for arg in zip(res_val,res_ind):
            #print(arg)
            print([(key,val,arg[0]) for key,val in word_to_ix.items() if val == arg[1]])

    def freeze_layer(self,layer):
        for name,child in model.named_children():
            print(name,child)
            if(name == layer):
                for names,params in child.named_parameters():
                    print(names,params)
                    print(params.size())
                    params.requires_grad= False

    def print_layer_parameters(self):
        for name,child in model.named_children():
                print(name,child)
                for names,params in child.named_parameters():
                    print(names,params)
                    print(params.size())

    def write_embedding_to_file(self,filename):
        for i in self.embeddings.parameters():
            weights = i.data.numpy()
        np.save(filename,weights)

# 1.4 读取数据

In [8]:
# 我们应该标记输入，但是现在我们将忽略它建立一个元组列表。 
# 每个元组是（[word_i-2，word_i-1]，目标词）

# 可以通过读取文件的方式进行输入
#test_sentence = read_data('https://www.gutenberg.org/files/57884/57884-0.txt')
ngrams = []
for i in range(len(test_sentence) - CONTEXT_SIZE):
    tup = [test_sentence[j] for j in np.arange(i , i + CONTEXT_SIZE) ]
    ngrams.append((tup,test_sentence[i + CONTEXT_SIZE]))
    
# 打印前3个，以便您可以看到它们的外观
#print(ngrams)
vocab = set(test_sentence)
print("词汇长度",len(vocab))
word_to_ix = {word: i for i, word in enumerate(vocab)}

词汇长度 195


# 1.5 训练模型

In [9]:
losses = []
loss_function = nn.NLLLoss()
model = CBOWModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

# 冻结嵌入层
#model.freeze_layer('embeddings')

for epoch in range(40):
    total_loss = 0
    #------- 嵌入层也在这里接受培训 ----#
    #lookup_tensor = torch.tensor([word_to_ix["poor"]], dtype=torch.long)
    #hello_embed = model.embeddings(lookup_tensor)
    #print(hello_embed)
    # -----------------------------------------------------#

    for context, target in ngrams:

        # 步骤1.准备要传递给模型的输入（即，将单词转换为整数索引并将其包装在张量中）
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        #print("Context id",context_idxs)

        # 步骤2.回想一下，割炬“累积”了渐变。 在传递之前新的实例，您需要将旧的梯度归零个实例
        model.zero_grad()

        # 步骤3.运行正向传递，获取下一个单词的对数概率
        log_probs = model(context_idxs)
        #print(log_probs)

        # 步骤4.计算损失函数。 （同样，Torch希望将目标单词包裹在张量中）
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
        #print(loss)

        # 步骤5.进行向后传递并更新渐变
        loss.backward()
        optimizer.step()

        # 通过调用tensor.item（）从1元素张量中获取Python编号
        total_loss += loss.item()
    print(total_loss)
    losses.append(total_loss)
#print(losses)  # 在训练数据上，每次迭代的损失都减少了！

#打印模型层参数
#model.print_layer_parameters()

1350.1079983711243
1341.4893674850464
1332.985544204712
1324.5890173912048
1316.2864503860474
1308.0782823562622
1299.9648191928864
1291.9407658576965
1284.0011630058289
1276.141842842102
1268.3641951084137
1260.678885936737
1253.0879261493683
1245.590512752533
1238.1786677837372
1230.8533253669739
1223.6161201000214
1216.4597299098969
1209.3816055059433
1202.3745067119598
1195.42345225811
1188.522690296173
1181.6791365146637
1174.8736196756363
1168.0926790237427
1161.3330707550049
1154.590341925621
1147.8491702079773
1141.1046557426453
1134.355996966362
1127.5902042388916
1120.802934885025
1113.9888179302216
1107.1450585126877
1100.2677028179169
1093.3541667461395
1086.4040232896805
1079.4086164236069
1072.3693451881409
1065.2769403457642


# 1.6 保存训练好的模型

In [10]:
model.write_embedding_to_file('embeddings.npy')

# 1.7 使用模型进行预测

In [21]:
#根据n个上下文词预测下一个词
model.predict(['of','all','and'])
cluster_embeddings('embeddings.npy',1)

[('the', 177, tensor(-3.4485, grad_fn=<SelectBackward>))]
[('and', 131, tensor(-3.7598, grad_fn=<SelectBackward>))]
[('in', 50, tensor(-3.8085, grad_fn=<SelectBackward>))]
50
in


## 2 SKIP-GRAM
![略图.png](https://upload.wikimedia.org/wikipedia/commons/9/95/Skip-gram.png)

概要：利用单词预测文本

以下是SKIP-GRAM算法的Pytorch实现。

# 2.1 导入全部的依赖包

In [32]:
# 作者: Srijith Rajamohan

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import urllib.request
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import word_tokenize
import sklearn
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances

# 2.2 初始化

In [33]:
torch.manual_seed(1)

CONTEXT_SIZE = 3
EMBEDDING_DIM = 10

test_sentence = """Empathy for the poor may not come easily to people who never experienced it. They may blame the victims and insist their predicament can be overcome through determination and hard work.
But they may not realize that extreme poverty can be psychologically and physically incapacitating — a perpetual cycle of bad diets, health care and education exacerbated by the shaming and self-fulfilling prophecies that define it in the public imagination.
Gordon Parks — perhaps more than any artist — saw poverty as “the most savage of all human afflictions” and realized the power of empathy to help us understand it. It was neither an abstract problem nor political symbol, but something he endured growing up destitute in rural Kansas and having spent years documenting poverty throughout the world, including the United States.
That sensitivity informed “Freedom’s Fearful Foe: Poverty,” his celebrated photo essay published in Life magazine in June 1961. He took readers into the lives of a Brazilian boy, Flavio da Silva, and his family, who lived in the ramshackle Catacumba favela in the hills outside Rio de Janeiro. These stark photographs are the subject of a new book, “Gordon Parks: The Flavio Story” (Steidl/The Gordon Parks Foundation), which accompanies a traveling exhibition co-organized by the Ryerson Image Centre in Toronto, where it opens this week, and the J. Paul Getty Museum. Edited with texts by the exhibition’s co-curators, Paul Roth and Amanda Maddox, the book also includes a recent interview with Mr. da Silva and essays by Beatriz Jaguaribe, Maria Alice Rezende de Carvalho and Sérgio Burgi.
""".split()

# 2.3 函数

In [34]:
def get_key(word_id):
    for key,val in word_to_ix.items():
        if(val == word_id):
            print(key)


def cluster_embeddings(filename,nclusters):
    X = np.load(filename)
    kmeans = KMeans(n_clusters=nclusters, random_state=0).fit(X)
    center = kmeans.cluster_centers_
    distances = euclidean_distances(X,center)

    for i in np.arange(0,distances.shape[1]):
        word_id = np.argmin(distances[:,i])
        print(word_id)
        get_key(word_id)

def read_data(file_path):
    tokenizer = RegexpTokenizer(r'\w+')
    data = urllib.request.urlopen(file_path)
    data = data.read().decode('utf8')
    tokenized_data = word_tokenize(data)
    stop_words = set(stopwords.words('english'))
    stop_words.update(['.',',',':',';','(',')','#','--','...','"'])
    cleaned_words = [ i for i in tokenized_data if i not in stop_words ]
    return(cleaned_words)

class SkipgramModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(SkipgramModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.linear2 = nn.Linear(128, context_size * vocab_size)
        #self.parameters['context_size'] = context_size

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))  # -1表示从数据大小推断出该索引的大小
        #print(np.mean(np.mean(self.linear2.weight.data.numpy())))
        out1 = F.relu(self.linear1(embeds)) # 第一层的输出
        out2 = self.linear2(out1)           # 第二层的输出
        #print(embeds)
        log_probs = F.log_softmax(out2, dim=1).view(CONTEXT_SIZE,-1)
        return log_probs

    def predict(self,input):
        context_idxs = torch.tensor([word_to_ix[input]], dtype=torch.long)
        res = self.forward(context_idxs)
        res_arg = torch.argmax(res)
        res_val, res_ind = res.sort(descending=True)
        indices = [res_ind[i][0] for i in np.arange(0,3)]
        for arg in indices:
            print( [ (key, val) for key,val in word_to_ix.items() if val == arg ])


    def freeze_layer(self,layer):
        for name,child in model.named_children():
            print(name,child)
            if(name == layer):
                for names,params in child.named_parameters():
                    print(names,params)
                    print(params.size())
                    params.requires_grad= False

    def print_layer_parameters(self):
        for name,child in model.named_children():
                print(name,child)
                for names,params in child.named_parameters():
                    print(names,params)
                    print(params.size())

    def write_embedding_to_file(self,filename):
        for i in self.embeddings.parameters():
            weights = i.data.numpy()
        np.save(filename,weights)

# 2.4 读取数据

In [35]:
# 我们应该标记输入，但是现在我们将忽略它
# 建立一个元组列表。 每个元组是（[word_i-2，word_i-1]，目标词）

#test_sentence = read_data('https://www.gutenberg.org/files/57884/57884-0.txt')

ngrams = []
for i in range(len(test_sentence) - CONTEXT_SIZE):
    tup = [test_sentence[j] for j in np.arange(i + 1 , i + CONTEXT_SIZE + 1) ]
    ngrams.append((test_sentence[i],tup))

# 打印前3个，以便您可以看到它们的外观
#print(ngrams)


vocab = set(test_sentence)
print("词汇长度",len(vocab))
word_to_ix = {word: i for i, word in enumerate(vocab)}

词汇长度 195


# 2.5 训练模型

In [36]:
losses = []
loss_function = nn.NLLLoss()
model = SkipgramModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

# 冻结嵌入层
#model.freeze_layer('embeddings')

for epoch in range(550):
    total_loss = 0
    #-------在这里也对嵌入层进行了训练----#
    #lookup_tensor = torch.tensor([word_to_ix["poor"]], dtype=torch.long)
    #hello_embed = model.embeddings(lookup_tensor)
    #print(hello_embed)
    # -----------------------------------------------------#

    model.predict('psychologically')

    for context, target in ngrams:

        # 步骤1.准备要传递给模型的输入（即，将单词转换为整数索引并将其包装在张量中）
        #print(context,target)


        context_idxs = torch.tensor([word_to_ix[context]], dtype=torch.long)
        #print("Context id",context_idxs)

        # 步骤2. 回想一下，火炬“累积”了渐变。 在传递之前新的实例，您需要将旧的梯度归零个实例
        model.zero_grad()


        # 步骤3.运行正向传递，获取下一个单词的对数概率
        log_probs = model(context_idxs)
        #print(log_probs)

        # 步骤4.计算损失函数。 （同样，火炬希望目标包裹在张量中的单词）
        target_list = torch.tensor([word_to_ix[w] for w in target], dtype=torch.long)
        loss = loss_function(log_probs, target_list)
        #print(loss)

        # 步骤5.进行向后传递并更新渐变
        loss.backward()
        optimizer.step()

        # 通过调用tensor.item（）从1元素张量中获取Python编号
        total_loss += loss.item()
    print(total_loss)
    losses.append(total_loss)
#print(losses)  # 损失减少了每次迭代训练数据！

#打印模型层参数
#model.print_layer_parameters()

[('He', 160)]
[('Museum.', 92)]
[('Mr.', 28)]
1644.1293816566467
[('He', 160)]
[('Museum.', 92)]
[('Mr.', 28)]
1641.444179534912
[('He', 160)]
[('Museum.', 92)]
[('Mr.', 28)]
1638.7803282737732
[('He', 160)]
[('Museum.', 92)]
[('Mr.', 28)]
1636.1370191574097
[('He', 160)]
[('Museum.', 92)]
[('Mr.', 28)]
1633.5124254226685
[('He', 160)]
[('Museum.', 92)]
[('Mr.', 28)]
1630.904902458191
[('He', 160)]
[('Museum.', 92)]
[('Mr.', 28)]
1628.3143224716187
[('He', 160)]
[('Museum.', 92)]
[('Mr.', 28)]
1625.7395176887512
[('He', 160)]
[('Museum.', 92)]
[('may', 158)]
1623.1799116134644
[('He', 160)]
[('Museum.', 92)]
[('may', 158)]
1620.634804725647
[('the', 177)]
[('Museum.', 92)]
[('may', 158)]
1618.1036400794983
[('the', 177)]
[('Museum.', 92)]
[('may', 158)]
1615.5848317146301
[('the', 177)]
[('Museum.', 92)]
[('may', 158)]
1613.0784277915955
[('the', 177)]
[('of', 166)]
[('may', 158)]
1610.5840029716492
[('the', 177)]
[('of', 166)]
[('may', 158)]
1608.1006507873535
[('the', 177)]
[('of', 1

# 2.6 保存训练好的模型

In [37]:
model.write_embedding_to_file('embeddings_skipgrams.npy')

# 2.7 使用模型进行预测

In [42]:
# 根据n个上下文词预测下一个词
model.predict('psychologically')
cluster_embeddings('embeddings_skipgrams.npy',6)

[('and', 131)]
[('physically', 106)]
[('incapacitating', 22)]
40
years
87
they
85
empathy
88
Centre
48
psychologically
145
took
